In [1]:
import json
import sys
from mrtparse import *
import networkx as nx
import matplotlib.pyplot as plt

import os
import requests
import gzip
import shutil
import openai

from neo4j import GraphDatabase

In [2]:
import os
name = 'OPENAI_API_KEY'

key_location = "/Users/kiranjyothisheena/Documents/Kiran_Files/Capstone/OpenAPI_Key"
key_file = open(key_location, "r")
openapi_key = key_file.readline()

#secret = getpass.getpass("Enter the OpenAI API Key")
os.environ[name] = openapi_key
print(f"Have successfully set up {name} as an environment variable.")

Have successfully set up OPENAI_API_KEY as an environment variable.


In [3]:
# Define your API key (keep it secret!)
openai.api_key = openapi_key
API_KEY = openapi_key

In [4]:
from openai import OpenAI
client = OpenAI()

In [5]:
# Connect to the Neo4j database
uri = "neo4j+s://28e5b287.databases.neo4j.io"  # Change to match your Neo4j server settings
username = "neo4j"      # Change to your Neo4j username
password = "z9J3DPCCxGYELn99XDdFbFIBnWOwR5fdn4MiG_Nvdck"      # Change to your Neo4j password

# Create a Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password))

## Get Schema of graph

In [6]:
from typing import Any, Dict, List, Optional

def neo4j_query(query: str, params: dict = {}) -> List[Dict[str, Any]]:

    """Query Neo4j database."""
    from neo4j.exceptions import CypherSyntaxError

    with driver.session(database="neo4j") as session:
        try:
            data = session.run(query, params)
            return [r.data() for r in data]
        except CypherSyntaxError as e:
            raise ValueError(f"Generated Cypher Statement is not valid\n{e}")

In [9]:
def refresh_schema() -> str:
    """
    Refreshes the Neo4j graph schema information.
    """
    
    
    node_properties_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
    WITH label AS nodeLabels, collect({property:property, type:type}) AS properties
    RETURN {labels: nodeLabels, properties: properties} AS output

    """

    rel_properties_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE NOT type = "RELATIONSHIP" AND elementType = "relationship"
    WITH label AS nodeLabels, collect({property:property, type:type}) AS properties
    RETURN {type: nodeLabels, properties: properties} AS output
    """

    rel_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE type = "RELATIONSHIP" AND elementType = "node"
    UNWIND other AS other_node
    RETURN {start: label, type: property, end: toString(other_node)} AS output
    """
    
    
    node_properties = [el["output"] for el in neo4j_query(node_properties_query)]
    rel_properties = [el["output"] for el in neo4j_query(rel_properties_query)]
    relationships = [el["output"] for el in neo4j_query(rel_query)]

    structured_schema = {
        "node_props": {el["labels"]: el["properties"] for el in node_properties},
        "rel_props": {el["type"]: el["properties"] for el in rel_properties},
        "relationships": relationships,
    }
    schema_str = f"""
    Node properties are the following:
    {node_properties}
    Relationship properties are the following:
    {rel_properties}
    The relationships are the following:
    {[f"(:{el['start']})-[:{el['type']}]->(:{el['end']})" for el in relationships]}
    """
    
    return schema_str

In [10]:
schema_str = refresh_schema()

In [11]:
print(schema_str)


    Node properties are the following:
    [{'labels': 'Node', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'as_name', 'type': 'STRING'}, {'property': 'org_name', 'type': 'STRING'}, {'property': 'country', 'type': 'STRING'}, {'property': 'upstream', 'type': 'STRING'}, {'property': 'downstream', 'type': 'STRING'}, {'property': 'rank', 'type': 'STRING'}, {'property': 'city', 'type': 'STRING'}, {'property': 'state', 'type': 'STRING'}]}, {'labels': 'Node_2', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'as_name', 'type': 'STRING'}, {'property': 'org_name', 'type': 'STRING'}, {'property': 'country', 'type': 'STRING'}, {'property': 'upstream', 'type': 'STRING'}, {'property': 'downstream', 'type': 'STRING'}, {'property': 'rank', 'type': 'STRING'}, {'property': 'city', 'type': 'STRING'}, {'property': 'state', 'type': 'STRING'}]}]
    Relationship properties are the following:
    []
    The relationships are the following:
    ['(:Node)-[:CONNECTS]->(

## Prompt to get Cypher query

In [12]:
import openai

def get_gpt3_response(curr_schema, question, api_key, model="gpt-3.5-turbo"):
    """
    Send a request to the OpenAI Chat API and get a response from the model.
    
    Args:
        prompt (str): The user's question.
        context (str): Contextual information to be passed to the model.
        api_key (str): Your OpenAI API key.
        model (str): The model version to use, default is "gpt-3.5-turbo".
    
    Returns:
        str: The model's response.
    """
    
    system_prompt = """
    You are a useful assistant
    """
    
    
    new_context =  """
    
    "Human: Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided 
    relationship types and properties in the schema.\nDo not use any other relationship types or 
    properties that are not provided.\nSchema:\nNode properties are the following: \n 
    {}\n
    Note: Do not include any explanations or apologies in your responses.\n
    Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    \nDo not include any text except the generated Cypher statement.
    \n\nThe question is:\n\n{}".
    """.format(curr_schema,question)
    
    # Create the full prompt by combining the system prompt, context, and the user question
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": new_context}
    ]
    
    ## Use the OpenAI Python client to send the request
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    
    # Extract the response text and return
    return response.choices[0].message.content
    #return response.choices[0].message['content']


In [14]:
# Example usage
question = "How many path exists from node 2519 to node 10026 for nodes of type node_2?"
response = get_gpt3_response(schema_str, question,  API_KEY, model = "gpt-3.5-turbo")
print(response)

MATCH path = (n:Node_2 {id: '2519'})-[:CONNECTS_2*]->(m:Node_2 {id: '10026'})
RETURN count(path) as path_count


## Executing the query in neo4j

In [15]:
cypher_res = neo4j_query(response)
print(cypher_res)

[{'path_count': 7}]


## Making the second call to LLM

In [17]:
import openai

def get_gpt3_response_2(prompt, question, api_key, model="gpt-3.5-turbo"):
    """
    Send a request to the OpenAI Chat API and get a response from the model.
    
    Args:
        prompt (str): The user's question.
        context (str): Contextual information to be passed to the model.
        api_key (str): Your OpenAI API key.
        model (str): The model version to use, default is "gpt-3.5-turbo".
    
    Returns:
        str: The model's response.
    """
    
    system_prompt = """
    You are a useful assistant
    """
    
    
    new_context =  """
    
    "Human: You are an assistant that helps to form nice and human understandable answers
    .\nThe information part contains the provided information that you must use to construct an answer.
    \nThe provided information is authoritative, you must never doubt it or try to use your internal knowledge
    to correct it.\nMake the answer sound as a response to the question. 
    Do not mention that you based the result on the given information.\n
    If the provided information is empty, say that you don't know the answer.
    \nInformation:\n{}\n\nQuestion: \n]{}\n\nHelpful Answer:"
    """.format(prompt,question)
    
    # Create the full prompt by combining the system prompt, context, and the user question
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": new_context}
    ]
    
    ## Use the OpenAI Python client to send the request
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    
    # Extract the response text and return
    return response.choices[0].message.content



In [18]:
# Example usage
question = "How many path exists from node 2519 to node 10026 for nodes of type node_2?"
response_2 = get_gpt3_response_2(cypher_res, question,  API_KEY, model = "gpt-3.5-turbo")
print(response_2)

There are 7 paths that exist from node 2519 to node 10026 for nodes of type node_2.


## Putting it all together

In [9]:
import openai

from typing import Any, Dict, List, Optional

def neo4j_query(query: str, params: dict = {}) -> List[Dict[str, Any]]:

    """Query Neo4j database."""
    from neo4j.exceptions import CypherSyntaxError

    with driver.session(database="neo4j") as session:
        try:
            data = session.run(query, params)
            return [r.data() for r in data]
        except CypherSyntaxError as e:
            raise ValueError(f"Generated Cypher Statement is not valid\n{e}")
            
def refresh_schema() -> str:
    """
    Refreshes the Neo4j graph schema information.
    """
    
    
    node_properties_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
    WITH label AS nodeLabels, collect({property:property, type:type}) AS properties
    RETURN {labels: nodeLabels, properties: properties} AS output

    """

    rel_properties_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE NOT type = "RELATIONSHIP" AND elementType = "relationship"
    WITH label AS nodeLabels, collect({property:property, type:type}) AS properties
    RETURN {type: nodeLabels, properties: properties} AS output
    """

    rel_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE type = "RELATIONSHIP" AND elementType = "node"
    UNWIND other AS other_node
    RETURN {start: label, type: property, end: toString(other_node)} AS output
    """
    
    
    node_properties = [el["output"] for el in neo4j_query(node_properties_query)]
    rel_properties = [el["output"] for el in neo4j_query(rel_properties_query)]
    relationships = [el["output"] for el in neo4j_query(rel_query)]

    structured_schema = {
        "node_props": {el["labels"]: el["properties"] for el in node_properties},
        "rel_props": {el["type"]: el["properties"] for el in rel_properties},
        "relationships": relationships,
    }
    schema_str = f"""
    Node properties are the following:
    {node_properties}
    Relationship properties are the following:
    {rel_properties}
    The relationships are the following:
    {[f"(:{el['start']})-[:{el['type']}]->(:{el['end']})" for el in relationships]}
    """
    
    return schema_str

def get_gpt3_response(curr_schema, question, api_key, model="gpt-3.5-turbo"):
    """
    Send a request to the OpenAI Chat API and get a response from the model.
    
    Args:
        prompt (str): The user's question.
        context (str): Contextual information to be passed to the model.
        api_key (str): Your OpenAI API key.
        model (str): The model version to use, default is "gpt-3.5-turbo".
    
    Returns:
        str: The model's response.
    """
    
    system_prompt = """
    You are a usefule assistant but do not add any extra information or words unless specified
    """
    
    
    new_context =  """
    
    "Human: Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided 
    relationship types and properties in the schema.\nDo not use any other relationship types or 
    properties that are not provided.\nSchema:\nNode properties are the following: \n 
    {}\n
    Note: Do not include any explanations or apologies in your responses.\n
    Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    \nDo not include any text except the generated Cypher statement.
    \n\nThe question is:\n\n{}".
    """.format(curr_schema,question)
    
    # Create the full prompt by combining the system prompt, context, and the user question
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": new_context}
    ]
    
    ## Use the OpenAI Python client to send the request
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    
    # Extract the response text and return
    return response.choices[0].message.content
    #return response.choices[0].message['content']


def get_gpt3_response_2(prompt, question, api_key, model="gpt-3.5-turbo"):
    """
    Send a request to the OpenAI Chat API and get a response from the model.
    
    Args:
        prompt (str): The user's question.
        context (str): Contextual information to be passed to the model.
        api_key (str): Your OpenAI API key.
        model (str): The model version to use, default is "gpt-3.5-turbo".
    
    Returns:
        str: The model's response.
    """
    
    system_prompt = """
    You are a useful assistant
    """
    
    
    new_context =  """
    
    "Human: You are an assistant that helps to form nice and human understandable answers
    .\nThe information part contains the provided information that you must use to construct an answer.
    \nThe provided information is authoritative, you must never doubt it or try to use your internal knowledge
    to correct it.\nMake the answer sound as a response to the question. 
    Do not mention that you based the result on the given information.\n
    Mention all node information using the id number.
    If the provided information is empty, say that you don't know the answer.
    \nInformation:\n{}\n\nQuestion: \n]{}\n\nHelpful Answer:"
    """.format(prompt,question)
    
    # Create the full prompt by combining the system prompt, context, and the user question
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": new_context}
    ]
    
    ## Use the OpenAI Python client to send the request
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    
    # Extract the response text and return
    return response.choices[0].message.content




def lang_chain_custom(question,driver):
    
    curr_schema = refresh_schema()  
    
    ## Getting Cypher query
    response = get_gpt3_response(curr_schema, question,  API_KEY, model = "gpt-3.5-turbo")
    print('Cypher Query is',response)
    
    ## Interacting with neo4j
    cypher_response = neo4j_query(response)
    print('Neo4j answer is',cypher_response)
    
    ## Obtaining final answer
    response_2 = get_gpt3_response_2(cypher_response, question,  API_KEY, model = "gpt-3.5-turbo")
    
    return response_2

In [10]:
question = "How many path exists from node 2519 to node 10026 for nodes of type node_2?"
response = lang_chain_custom(question,driver)
print(response)

Cypher Query is MATCH p=(:Node_2 {id: '2519'})-[:CONNECTS_2*]->(:Node_2 {id: '10026'})
RETURN count(p)
Neo4j answer is [{'count(p)': 7}]
There are 7 paths that exist from node 2519 to node 10026 for nodes of type node_2.


In [7]:
question = "Return all paths from node 4651 to node 1136 for nodes of type node_2?"
response = lang_chain_custom(question,driver)
print(response)

Cypher Query is MATCH path = (start:Node_2 {id: '4651'})-[:CONNECTS_2*]->(end:Node_2 {id: '1136'})
RETURN path
Neo4j answer is [{'path': [{'country': 'AU', 'upstream': '9', 'downstream': '43', 'rank': '11319', 'id': '4651', 'org_name': 'CAT Telecom Public Company Limited', 'as_name': 'THAI-GATEWAY'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '1', 'downstream': '0', 'rank': '56868', 'id': '9737', 'org_name': 'TOT Public Company Limited', 'as_name': 'TOTNET-TH-AS-AP'}, 'CONNECTS_2', {'country': 'NL', 'upstream': '3', 'downstream': '25', 'rank': '108', 'id': '1136', 'org_name': 'KPN B.V.', 'as_name': 'KPN'}]}, {'path': [{'country': 'AU', 'upstream': '9', 'downstream': '43', 'rank': '11319', 'id': '4651', 'org_name': 'CAT Telecom Public Company Limited', 'as_name': 'THAI-GATEWAY'}, 'CONNECTS_2', {'country': 'NL', 'upstream': '3', 'downstream': '25', 'rank': '108', 'id': '1136', 'org_name': 'KPN B.V.', 'as_name': 'KPN'}]}]
Node 4651 has two paths to node 1136 for nodes of type node_2. 

P

In [11]:
question = "Return all paths from node 2519 to node 10026 for nodes of type node_2?"
response = lang_chain_custom(question,driver)
print(response)

Cypher Query is MATCH path = (start:Node_2 {id: '2519'})-[:CONNECTS_2*]->(end:Node_2 {id: '10026'})
RETURN path
Neo4j answer is [{'path': [{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '2', 'downstream': '225', 'rank': '61', 'id': '1221', 'org_name': 'Telstra Corporation Limited', 'as_name': 'ASN-TELSTRA'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '2', 'downstream': '0', 'rank': '9740', 'id': '38803', 'org_name': 'Wirefreebroadband Pty Ltd', 'as_name': 'WPL-AS-AP'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '8', 'downstream': '437', 'rank': '920', 'id': '4826', 'org_name': 'VOCUS PTY LTD', 'as_name': 'VOCUS-BACKBONE-AS'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '3', 'downstream': '193', 'rank': '3029', 'id': '4608', 'org_name': 'Asia Pacific Network Information Centre', 'as_name': 'APNIC-SERVICES'}, 'CONNECTS_2', {'country': 'AU', 'id': '10026', 'org_name': 'Telstra Interna

## Path questions

In [12]:
question = "Does a path exist from node 56203 to node 38803 for nodes of type node? If so what is the path?"
response = lang_chain_custom(question,driver)
print(response)

Cypher Query is MATCH path = (n:Node {id: '56203'})-[:CONNECTS*]->(m:Node {id: '38803'})
RETURN path
Neo4j answer is [{'path': [{'country': 'AU', 'id': '56203', 'org_name': 'Wirefreebroadband Pty Ltd', 'as_name': 'Gtelecom-AUSTRALIA'}, 'CONNECTS', {'country': 'AU', 'upstream': '2', 'downstream': '0', 'rank': '9740', 'id': '38803', 'org_name': 'Wirefreebroadband Pty Ltd', 'as_name': 'WPL-AS-AP'}]}]
Yes, a path exists from node 56203 to node 38803. The path is as follows:
56203 (Wirefreebroadband Pty Ltd - Gtelecom-AUSTRALIA) -> 38803 (Wirefreebroadband Pty Ltd - WPL-AS-AP)


In [13]:
question = "Does a path exist from node 2519 to node 18144 for nodes of type node? If so what is the path?"
response = lang_chain_custom(question,driver)
print(response)

Cypher Query is MATCH path=(n:Node {id: "2519"})-[:CONNECTS*]->(m:Node {id: "18144"})
RETURN path
Neo4j answer is []
I'm sorry, but as there is no provided information, I don't have the necessary data to determine if a path exists from node 2519 to node 18144 for nodes of type node, or what the path might be.


In [7]:
import openai

from typing import Any, Dict, List, Optional

def neo4j_query(query: str, params: dict = {}) -> List[Dict[str, Any]]:

    """Query Neo4j database."""
    from neo4j.exceptions import CypherSyntaxError

    with driver.session(database="neo4j") as session:
        try:
            data = session.run(query, params)
            return [r.data() for r in data]
        except CypherSyntaxError as e:
            raise ValueError(f"Generated Cypher Statement is not valid\n{e}")
            
def refresh_schema() -> str:
    """
    Refreshes the Neo4j graph schema information.
    """
    
    
    node_properties_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
    WITH label AS nodeLabels, collect({property:property, type:type}) AS properties
    RETURN {labels: nodeLabels, properties: properties} AS output

    """

    rel_properties_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE NOT type = "RELATIONSHIP" AND elementType = "relationship"
    WITH label AS nodeLabels, collect({property:property, type:type}) AS properties
    RETURN {type: nodeLabels, properties: properties} AS output
    """

    rel_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE type = "RELATIONSHIP" AND elementType = "node"
    UNWIND other AS other_node
    RETURN {start: label, type: property, end: toString(other_node)} AS output
    """
    
    
    node_properties = [el["output"] for el in neo4j_query(node_properties_query)]
    rel_properties = [el["output"] for el in neo4j_query(rel_properties_query)]
    relationships = [el["output"] for el in neo4j_query(rel_query)]

    structured_schema = {
        "node_props": {el["labels"]: el["properties"] for el in node_properties},
        "rel_props": {el["type"]: el["properties"] for el in rel_properties},
        "relationships": relationships,
    }
    schema_str = f"""
    Node properties are the following:
    {node_properties}
    Relationship properties are the following:
    {rel_properties}
    The relationships are the following:
    {[f"(:{el['start']})-[:{el['type']}]->(:{el['end']})" for el in relationships]}
    """
    
    return schema_str

def get_gpt3_response(curr_schema, question, api_key, model="gpt-3.5-turbo"):
    """
    Send a request to the OpenAI Chat API and get a response from the model.
    
    Args:
        prompt (str): The user's question.
        context (str): Contextual information to be passed to the model.
        api_key (str): Your OpenAI API key.
        model (str): The model version to use, default is "gpt-3.5-turbo".
    
    Returns:
        str: The model's response.
    """
    
    system_prompt = """
    You are a usefule assistant but do not add any extra information or words unless specified
    """
    
    
    new_context =  """
    
    "Human: Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided 
    relationship types and properties in the schema.\nDo not use any other relationship types or 
    properties that are not provided.\nSchema:\nNode properties are the following: \n 
    {}\n
    Note: Do not include any explanations or apologies in your responses.\n
    Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    \nDo not include any text except the generated Cypher statement.
    \n\nThe question is:\n\n{}".
    """.format(curr_schema,question)
    
    # Create the full prompt by combining the system prompt, context, and the user question
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": new_context}
    ]
    
    ## Use the OpenAI Python client to send the request
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    
    # Extract the response text and return
    return response.choices[0].message.content
    #return response.choices[0].message['content']


def get_gpt3_response_2(prompt, question, api_key, model="gpt-3.5-turbo"):
    """
    Send a request to the OpenAI Chat API and get a response from the model.
    
    Args:
        prompt (str): The user's question.
        context (str): Contextual information to be passed to the model.
        api_key (str): Your OpenAI API key.
        model (str): The model version to use, default is "gpt-3.5-turbo".
    
    Returns:
        str: The model's response.
    """
    
    system_prompt = """
    You are a useful assistant
    """
    
    
    new_context =  """
    
    "Human: You are an assistant that helps to form nice and human understandable answers
    .\nThe information part contains the provided information that you must use to construct an answer.
    \nThe provided information is authoritative, you must never doubt it or try to use your internal knowledge
    to correct it.\nMake the answer sound as a response to the question. 
    Do not mention that you based the result on the given information.\n
    Mention all node information using the id number.
    If the question is to check whether a path exists or not and if the information is empty
    then return that path do not exist .\n   
    In all other cases, If the provided information is empty, say that you don't know the answer.\n
    \nInformation:\n{}\n\nQuestion: \n]{}\n\nHelpful Answer:"
    """.format(prompt,question)
    
    # Create the full prompt by combining the system prompt, context, and the user question
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": new_context}
    ]
    
    ## Use the OpenAI Python client to send the request
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    
    # Extract the response text and return
    return response.choices[0].message.content




def lang_chain_custom(question,driver):
    
    curr_schema = refresh_schema()  
    
    ## Getting Cypher query
    response = get_gpt3_response(curr_schema, question,  API_KEY, model = "gpt-3.5-turbo")
    print('Cypher Query is',response)
    
    ## Interacting with neo4j
    cypher_response = neo4j_query(response)
    print('Neo4j answer is',cypher_response)
    
    ## Obtaining final answer
    response_2 = get_gpt3_response_2(cypher_response, question,  API_KEY, model = "gpt-3.5-turbo")
    
    return response_2

In [8]:
question = "Does a path exist from node 2519 to node 18144 for nodes of type node? If so what is the path?"
response = lang_chain_custom(question,driver)
print(response)

Cypher Query is MATCH path = (n1:Node {id: 2519})-[:CONNECTS*]->(n2:Node {id: 18144})
RETURN path;
Neo4j answer is []
No, I'm sorry, I don't have any information to determine whether a path exists from node 2519 to node 18144 for nodes of type node. Since the information is empty, I cannot provide the path for you.


## Shortest Path

In [10]:
question = """What is the shortest path from node 2519 to node 10026 for nodes of type node? Refer all nodes
with respect to their Ids in the final answer""" 
response = lang_chain_custom(question,driver)
print(response)

Cypher Query is MATCH (n1:Node {id: "2519"}), (n2:Node {id: "10026"})
MATCH path = shortestPath((n1)-[:CONNECTS*]-(n2))
RETURN [node in nodes(path) | node.id] AS ShortestPath
Neo4j answer is [{'ShortestPath': ['2519', '10026']}]
The shortest path from node 2519 to node 10026 for nodes of type node is:
2519 -> 10026


In [11]:
question = """What is the shortest path from node 2519 to node 10026 for nodes of type node_2? Refer all nodes
with respect to their Ids in the final answer""" 
response = lang_chain_custom(question,driver)
print(response)

Cypher Query is MATCH p=shortestPath((n1:Node_2 {id: '2519'})-[:CONNECTS_2*]-(n2:Node_2 {id: '10026'}))
RETURN nodes(p) AS path_nodes, relationships(p) AS path_relationships, length(p) AS path_length


Neo4j answer is [{'path_nodes': [{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, {'country': 'AU', 'upstream': '2', 'downstream': '225', 'rank': '61', 'id': '1221', 'org_name': 'Telstra Corporation Limited', 'as_name': 'ASN-TELSTRA'}, {'country': 'AU', 'upstream': '3', 'downstream': '193', 'rank': '3029', 'id': '4608', 'org_name': 'Asia Pacific Network Information Centre', 'as_name': 'APNIC-SERVICES'}, {'country': 'AU', 'id': '10026', 'org_name': 'Telstra International Limited', 'as_name': 'TELSTRAGLOBAL'}], 'path_relationships': [({'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '2', 'downstream': '225', 'rank': '61', 'id': '122

In [15]:
question = """What is the out degree of node 10026? Refer all nodes
with respect to their Ids in the final answer""" 
response = lang_chain_custom(question,driver)
print(response)

Cypher Query is MATCH (n:Node)-[:CONNECTS]->()
WHERE n.id = '10026'
RETURN n.id, COUNT(*) AS outDegree
ORDER BY n.id
Neo4j answer is [{'n.id': '10026', 'outDegree': 4}]
The out degree of node 10026 is 4.
